In [ ]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img, img_to_array, array_to_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten,LeakyReLU
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D,Reshape,Conv2DTranspose
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random
import glob
import matplotlib.pyplot as plt
import warnings
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from keras.utils.vis_utils import plot_model
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy import vstack
from numpy.random import randn
from numpy.random import randint
from keras.datasets.mnist import load_data
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from matplotlib import pyplot
import zipfile

with zipfile.ZipFile("../input/generative-dog-images/all-dogs.zip","r") as zip_ref:
    zip_ref.extractall("/kaggle/temp/")
    
with zipfile.ZipFile("../input/generative-dog-images/Annotation.zip","r") as zip_ref:
    zip_ref.extractall("/kaggle/temp/")

In [ ]:
ROOT = '/kaggle/temp/'
epochs = 50
batch_size = 128
img_size = 64
noise_dim = 100
WEIGHT_INIT = tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.1)

In [ ]:
train = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input).flow_from_directory(directory=ROOT, target_size=(img_size,img_size), classes = ['all-dogs'],batch_size = 16)

In [ ]:
data_list = []
batch_index = 0

from tqdm import tqdm
for batch_index in tqdm(range(50)):
    data = train.next()
    data_list.append(data[0])

imgs = np.asarray(data_list)

In [ ]:
imgs = imgs.reshape(800,64,64,3)
print(imgs.shape)
imgs = (imgs-127.5)/127.5

In [ ]:
def get_real(n_samples,img_array = imgs):
  indexes = randint(0,img_array.shape[0],n_samples)
  reals = img_array[indexes]
  return reals

In [ ]:
#build generator and return the model
def generator():
  model = Sequential()
  model.add(Dense(int(img_size/8)*int(img_size/8)*512, use_bias=False, input_shape=(100,),kernel_initializer=WEIGHT_INIT))
  model.add(BatchNormalization())
  model.add(LeakyReLU())

  model.add(Reshape((int(img_size/8), int(img_size/8), 512)))
  

  model.add(Conv2DTranspose(256, (5, 5), strides=(2, 2), padding='same', use_bias=False,kernel_initializer=WEIGHT_INIT))
  model.add(BatchNormalization())
  model.add(LeakyReLU())
  model.add(Dropout(0.3))
  

  model.add(Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', use_bias=False,kernel_initializer=WEIGHT_INIT))
  model.add(BatchNormalization())
  model.add(LeakyReLU())
  model.add(Dropout(0.3))

  model.add(Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False,kernel_initializer=WEIGHT_INIT ))
  model.add(BatchNormalization())
  model.add(LeakyReLU())
  model.add(Dense(3,activation = 'tanh',use_bias = False,kernel_initializer=WEIGHT_INIT))
  opt = Adam(lr=1e-4)
  model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
  return model

In [ ]:
def discriminator():
  model = tf.keras.Sequential()
  model.add(Conv2D(64, (4, 4), strides=(2, 2), padding='same',
                                    input_shape=[img_size, img_size, 3],kernel_initializer=WEIGHT_INIT))
  model.add(BatchNormalization())
  model.add(LeakyReLU())
  model.add(Dropout(0.3))

  model.add(Conv2D(128, (4, 4), strides=(2, 2), padding='same',kernel_initializer=WEIGHT_INIT))
  model.add(BatchNormalization())
  model.add(LeakyReLU())
  model.add(Dropout(0.3)) 

  model.add(Conv2D(256, (4, 4), strides=(2, 2), padding='same',kernel_initializer=WEIGHT_INIT))
  model.add(BatchNormalization())
  model.add(LeakyReLU())
  model.add(Dropout(0.3))

  model.add(Flatten())
  model.add(Dense(1))
  opt = Adam(lr=1e-4)
  model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
  return model

In [ ]:
def GAN(d,g):
    model = Sequential()
    d.trainable = False
    model.add(g)
    model.add(d)
    opt = Adam(lr=1e-4)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model

In [ ]:
generator = generator()
discriminator = discriminator()
gan = GAN(discriminator,generator)

In [ ]:
for epoch in range(1000):
  with tf.GradientTape() as gt, tf.GradientTape() as dt:
    #generate with frozen generator
    noise = tf.random.normal(shape = (batch_size,noise_dim))
    generated_img = generator(noise)
    #mix it with real inputs
    real_img = get_real(batch_size)
    input,output = vstack((generated_img,real_img)), vstack((np.zeros((batch_size,1)),np.ones((batch_size,1))))
    #train discriminator
    d_loss,_ = discriminator.train_on_batch(x = input, y = output)
    #now train on the whole model
    noise = tf.random.normal(shape = (batch_size,noise_dim))

    result = np.ones(batch_size)
    g_loss = gan.train_on_batch(noise, result)
    print(f'epoch: {epoch}, d_loss: {d_loss}, g_loss: {g_loss}')

In [ ]:
generator.save('/g_model')

In [ ]:
from keras.models import load_model
from matplotlib import pyplot
model = load_model('/g_model')
noise = tf.random.normal(shape = (25,100))
x = model.predict(noise)
def save_plot(examples, n):
	# plot images
  for i in range(n * n):
    # define subplot
    pyplot.subplot(n, n, 1 + i)
    # turn off axis
    pyplot.axis('off')
    # plot raw pixel data
    print(examples)
    pyplot.imshow(examples[i, :, :, 0]*127.5+127.5)
  pyplot.show()
save_plot(x, 1)